# Librerias

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import MSTL
from datetime import datetime


import matplotlib.pyplot as plt
import seaborn as sbn
from datetime import datetime

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_privacy
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy


# Abrir datos 

## Datos Contugas

In [2]:
# abrir datos contugas

new1 = pd.read_excel('EICH106.xlsx')
new1.columns = ['VOLUMEN CORREGIDO', 'STD_VOLUME', 'ORIG_TEMPERATURE', 'TEMPERATURA','PRESION', 'ORIG_PRESSURE', 'VOLUMENSINCORREGIR', 'RAW_VOLUME', 'FECHAINICIAL']

#función que pone las fechas en el mismo formato
def cambiofecha(row):
    
    for i in range(len(row)):
        if isinstance(row.at[i, 'FECHAINICIAL'], str):
            row.at[i, 'FECHAINICIAL'] = pd.to_datetime(row.at[i, 'FECHAINICIAL']).strftime('%Y-%m-%d %H:%M:%S')  
        elif isinstance(row.at[i, 'FECHAINICIAL'], datetime):
            row.at[i, 'FECHAINICIAL'] = datetime.strptime(str(row.at[i, 'FECHAINICIAL']),'%Y-%d-%m %H:%M:%S')
            row.at[i, 'FECHAINICIAL'] = row.at[i, 'FECHAINICIAL'].strftime('%Y-%m-%d %H:%M:%S')

    return row

new1=cambiofecha(new1)

new1 = new1.set_index('FECHAINICIAL')
new1.index = pd.to_datetime(new1.index, format='%Y-%m-%d %H:%M:%S')


# Función que añade al dataframe la hora, dia de la semana, mes y dia del año.
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    #df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    #df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    #df['dayofmonth'] = df.index.day
    #df['weekofyear'] = df.index.isocalendar().week
    return df


new1 = create_features(new1)

datos1=new1[["PRESION", "TEMPERATURA", "VOLUMENSINCORREGIR", "hour", "dayofweek", "month", "dayofyear"]]


#función que elimina las anomalias 

def eliminar_anomalias(df1,Vol,VolMin,VolMax,Temp,TempMin,TempMax,Presion,PresMin,PresMax):

    df=df1.copy()

    if Vol == True:
        df["VOLUMENSINCORREGIR"]= np.where((df["VOLUMENSINCORREGIR"]<VolMin)|(df["VOLUMENSINCORREGIR"]>VolMax),df['VOLUMENSINCORREGIR'].shift(168),df['VOLUMENSINCORREGIR'])

    if Presion == True:
        df["PRESION"]= np.where((df["PRESION"]<PresMin)|(df["PRESION"]>PresMax),df['PRESION'].shift(168),df['PRESION'])

    if Temp == True:
        df["TEMPERATURA"]= np.where((df["TEMPERATURA"]<TempMin)|(df["TEMPERATURA"]>TempMax),df['TEMPERATURA'].shift(168),df['TEMPERATURA'])


    return df



datos1=eliminar_anomalias(datos1,   True,0,250,True,17,35,True,14,19) ## Falta Presion

# Crear funciones

## STL - DP

In [4]:
# crear función 

def STL_DP_S(datos,deltaf_p,b_p):

    mstl = MSTL(datos, periods=[24, 24 * 7], iterate=5, stl_kwargs={"seasonal_deg": 0,
                                                                            "inner_iter": 2,
                                                                            "outer_iter": 0})
    res = mstl.fit() # Use .fit() to perform and return the decomposition
    #ax = res.plot()
    #plt.tight_layout()


    res.trend

    tendencia = res.trend
    seasonal = res.seasonal
    residual = res.resid


    tendenciaFourier = np.fft.fft(tendencia)

    # Generar el ruido Laplaciano y aplicarlo a los coeficientes de Fourier
    b = b_p
    deltaf = deltaf_p
    epsilon = deltaf/ b

    # loc = media, scale = b
    laplace = np.random.laplace(loc=0, scale=1/epsilon,size = tendenciaFourier.shape )

    #laplace_noise = np.random.laplace(loc=0, scale=b, size=tendenciaFourier.shape)
    perturbed_trend_dft = tendenciaFourier + laplace


    # 
    perturbed_trend = np.fft.ifft(perturbed_trend_dft).real


    # sacar datos de ruido
    DatosRuido = perturbed_trend + seasonal['seasonal_168'] + seasonal['seasonal_24'] + residual



    return DatosRuido,epsilon

## LSTM normal

In [ ]:
def LSTM_JD(datos,ventana,prediccion,fechas,nodos1,nodos2,paciencia,epocas,batch,activacion1,activacion2,activacion3):


    datos = datos.values.reshape(-1, 1)

    estandarizacion = MinMaxScaler().fit(datos)
    scaled_data = estandarizacion.transform(datos)


    # dividir en train, test
    X, y = [], []
    Xf,yf = [],[]

    for i in range(len(scaled_data) - ventana - prediccion):
        X.append(scaled_data[i:i+ventana])
        y.append(scaled_data[i+ventana:i+ventana+prediccion])

        Xf.append(fechas[i:i+ventana])
        yf.append(fechas[i+ventana:i+ventana+prediccion])

    X, y = np.array(X), np.array(y)
    Xf,yf = np.array(Xf),np.array(yf)



    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    fecha_X_train, fecha_X_test, fecha_y_train, fecha_y_test = train_test_split(Xf, yf, test_size=0.1, shuffle=False)


    model = Sequential()

    model.add(LSTM(nodos1,activation= activacion1, input_shape=(ventana,1)))
    model.add(Dense(nodos2, activation=activacion2))
    model.add(Dense(prediccion , activation=activacion3))


    model.compile(optimizer="Adam", loss='mse')
    early_stopping = EarlyStopping(monitor='loss', patience=paciencia, restore_best_weights=True)

    history = model.fit(X_train, y_train, epochs=epocas,validation_split = 0.2, verbose=1, batch_size=batch,shuffle = False, callbacks=[early_stopping])


    # guardar los archivo a usar en la carpeta 
    rutaAGuardar = f'Modelo {nodos1} - nodos1 - {nodos2} - nodos2 - {epocas} Epocas sin PD.keras'
    model.save(rutaAGuardar)


        
    y_hat = model.predict(X_test, verbose=1)
    y_hat = estandarizacion.inverse_transform(y_hat)

    y_test1 = y_test.reshape(-1, 1)

    y_test1 = estandarizacion.inverse_transform(y_test1)

    y_test1 = y_test1.reshape(-1,24,1)


    return y_hat,y_test1,fecha_y_test, history


## LSTM TF-P

In [4]:
def LSTM_TFP_JD(datos,ventana,prediccion,fechas,nodos1,nodos2,paciencia,epocas,batch,activacion1,activacion2,activacion3,norm_clip,ruido,microBatches,lr):
    

    datos = datos.values.reshape(-1, 1)

    estandarizacion = MinMaxScaler().fit(datos)
    scaled_data = estandarizacion.transform(datos)


    # dividir en train, test
    X, y = [], []
    Xf,yf = [],[]

    for i in range(len(scaled_data) - ventana - prediccion):
        X.append(scaled_data[i:i+ventana])
        y.append(scaled_data[i+ventana:i+ventana+prediccion])

        Xf.append(fechas[i:i+ventana])
        yf.append(fechas[i+ventana:i+ventana+prediccion])

    X, y = np.array(X), np.array(y)
    Xf,yf = np.array(Xf),np.array(yf)



    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    fecha_X_train, fecha_X_test, fecha_y_train, fecha_y_test = train_test_split(Xf, yf, test_size=0.1, shuffle=False)


    model = Sequential()

    model.add(LSTM(nodos1,activation= activacion1, input_shape=(ventana,1)))
    model.add(Dense(nodos2, activation=activacion2))
    model.add(Dense(prediccion , activation=activacion3))




    if batch % microBatches != 0:
        raise ValueError('Batch size should be an integer multiple of the number of microbatches')



    # agregar la privacidad diferencial en el optimizador 
    optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
        l2_norm_clip=norm_clip,
        noise_multiplier=ruido,
        num_microbatches=microBatches,
        learning_rate=lr)

    # Función de pérdida para regresión
    loss = tf.keras.losses.MeanSquaredError(reduction=tf.losses.Reduction.NONE)


    model.compile(optimizer=optimizer, loss=loss)

    early_stopping = EarlyStopping(monitor='loss', patience=paciencia, restore_best_weights=True)

    history = model.fit(X_train, y_train, epochs=epocas,validation_split = 0.2, verbose=1, batch_size=batch,shuffle = False, callbacks=[early_stopping])

    # guardar los archivo a usar en la carpeta 
    rutaAGuardar = f'Modelo {nodos1} - nodos1 - {nodos2} - nodos2 - {epocas} Epocas sin PD.keras'
    model.save(rutaAGuardar)


        
    y_hat = model.predict(X_test, verbose=1)
    y_hat = estandarizacion.inverse_transform(y_hat)

    y_test1 = y_test.reshape(-1, 1)

    y_test1 = estandarizacion.inverse_transform(y_test1)

    y_test1 = y_test1.reshape(-1,24,1)


    return y_hat,y_test1,fecha_y_test, history

# Correr Modelo

## Contugas

### Aplicar STL-DP a los datos

In [5]:
# Datos, DeltaF, B
DatosRuido,epsilon = STL_DP_S(datos1['TEMPERATURA'],1,100)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(datos1['TEMPERATURA'][:168].values)
plt.plot(DatosRuido.values[:168])
plt.grid()

### correr Red Neuronal LSTM Con Datos Perturbados y normales y correr TF-P

In [ ]:
# Parámetros

#datos = datos1['TEMPERATURA'].values.reshape(-1, 1)
ventana = 168
prediccion = 24
fechas = datos1.index
nodos1 = 100
nodos2 = 100
paciencia = 10
epocas = 20
batch = 32
activacion1 = "tanh"
activacion2 = "tanh"
activacion3 = "linear"

l2_norm_clip = 0
noise_multiplier = 10
num_microbatches = 4
learning_rate = 0.0001




In [ ]:
y_hatLSTM,y_testLSTM,fecha_y_testLSTM, historyLSTM = LSTM_JD(datos1['TEMPERATURA'],ventana,prediccion,fechas,nodos1,nodos2,paciencia,epocas,batch,activacion1,activacion2,activacion3)

In [ ]:
y_hat_STL_DP,y_test_STL_DP,fecha_y_test_STL_DP, history_STL_DP  = LSTM_JD(DatosRuido,ventana,prediccion,fechas,nodos1,nodos2,paciencia,epocas,batch,activacion1,activacion2,activacion3)

In [ ]:
y_hat_TF_P,y_test_TF_P,fecha_y_test_TF_P = LSTM_TFP_JD(datos1['TEMPERATURA'],ventana,prediccion,fechas,nodos1,nodos2,paciencia,epocas,batch,activacion1,activacion2,activacion3,l2_norm_clip,noise_multiplier,num_microbatches,learning_rate)



### Encontrar Errores

In [ ]:
a

In [ ]:
a

### Privacidad TF_P

In [ ]:
a

In [ ]:
a

## Función que haga todo junto

In [ ]:
def Tesis(datos,deltaf_p,b_p,ventana,prediccion,fechas,nodos1,nodos2,paciencia,epocas,batch,activacion1,activacion2,activacion3,l2_norm_clip,noise_multiplier,num_microbatches,learning_rate):

    # Realizar STL-DP
    DatosRuido,epsilon_STL_DP = STL_DP_S(datos,deltaf_p,b_p)

    # Realizar LSTM con datos sin ruido
    y_hatLSTM,y_testLSTM,fecha_y_testLSTM, historyLSTM = LSTM_JD(datos,ventana,prediccion,fechas,nodos1,nodos2,paciencia,epocas,batch,activacion1,activacion2,activacion3)
    
    # Realizar LSTM con datos STL_DP
    y_hat_STL_DP,y_test_STL_DP,fecha_y_test_STL_DP, history_STL_DP  = LSTM_JD(DatosRuido,ventana,prediccion,fechas,nodos1,nodos2,paciencia,epocas,batch,activacion1,activacion2,activacion3)

    # Realizar LSTM con TF_P
    y_hat_TF_P,y_test_TF_P,fecha_y_test_TF_P = LSTM_TFP_JD(datos,ventana,prediccion,fechas,nodos1,nodos2,paciencia,epocas,batch,activacion1,activacion2,activacion3,l2_norm_clip,noise_multiplier,num_microbatches,learning_rate)




    return